In [1]:
import random
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def get_pdf_data(user_res):
    
    PDF_file = "C:/Users/DELL/effectofsalinitandsodicityonplantgrowth.pdf"
    pages = convert_from_path(PDF_file, 500) 
    image_counter = 1
    for page in pages: 
        filename = "page_"+str(image_counter)+".jpg"
        page.save(filename, 'JPEG') 
        image_counter = image_counter + 1

    filelimit = image_counter-1
    outfile = "out_text.txt"
    #f = open(outfile, "a") 
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    corpus =''
    for i in range(1, filelimit + 1): 
        filename = "page_"+str(i)+".jpg"
        text = str(((pytesseract.image_to_string(Image.open(filename))))) 
        text = text.replace('-\n', '')     
        corpus+= text
    
    sent_tokens = nltk.sent_tokenize(corpus)
    return sent_tokens

In [3]:

## Method for getting the query from user and search in web.
def search_user_response(res):
    try: 
        from googlesearch import search 
    except ImportError:  
        print("No module named 'google' found") 
    #query = ''.join(res.split(' '))  
    for j in search(res, tld="co.in", num=10, stop=1, pause=2): 
        return (j) 


In [8]:

# Method for getting text from url when query is asked from user.
def get_web_data(res):
    inputs = []
    corpus =[]
    for inp in res.split():
        if inp in inputs:
            for i in range(1,4):
                url = search_user_response(res)
                page = requests.get(url,verify=False)
                if page.status_code==200:
                    soup = BeautifulSoup(page.content, 'html.parser')
                    for p in soup.find_all('p'):
                        corpus.append(p.get_text())
            page = requests.get(url,verify=False)
            soup = BeautifulSoup(page.content, 'html.parser')
            for p in soup.find_all('p'):
                corpus.append(p.get_text())
                    
    text =''
    #print(corpus)
    for ele in corpus:
        text+=ele
    #print('text:',text)
    sent_tokens = nltk.sent_tokenize(text) #Convert the text into a list of sentences

#Print the list of sentences
    #print(sent_tokens)
    
    return sent_tokens

In [9]:
#Tokenization

#Create a dictionary (key:value) pair to remove punctuations
remove_punct_dict = dict(  ( ord(punct),None) for punct in string.punctuation)

#Print the punctuations
print(string.punctuation)

#Print the dictionary
print(remove_punct_dict)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: None, 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 123: None, 124: None, 125: None, 126: None}


In [10]:
def LemNormalize(corpus):
  return nltk.word_tokenize(corpus.lower().translate(remove_punct_dict))

#Print the tokenization text
print(LemNormalize(corpus))

NameError: name 'corpus' is not defined

In [11]:
#Keyword Matching

#Greeting Inputs
GREETING_INPUTS = ["hi", "hello", "hola", "greetings", "wassup", "hey"]

#Greeting responses back to the user
GREETING_RESPONSES=["howdy", "hi", "hey", "what's good", "hello", "hey there"]

#Function to return a random greeting response to a users greeting
def greeting(sentence):
  #if the user's input is a greeting, then return a randomly chosen greeting response
  for word in sentence.split():
    if word.lower() in GREETING_INPUTS:
        return random.choice(GREETING_RESPONSES)

In [12]:
def response(user_response):
  

  #The users response / query
                
  user_response = user_response.lower() #Make the response lower case

  ###Print the users query/ response
  #print(user_response)

  #Set the chatbot response to an empty string
  robo_response = ''
  sent_tokens = get_pdf_data(user_response)

  #Append the users response to the sentence list
  sent_tokens.append(user_response)

  ###Print the sentence list after appending the users response
  #print(sent_tokens)

  #Create a TfidfVectorizer Object
  TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english')

  #Convert the text to a matrix of TF-IDF features
  tfidf = TfidfVec.fit_transform(sent_tokens)

  ###Print the TFIDF features
  #print(tfidf)
  #Get the measure of similarity (similarity scores)
  vals = cosine_similarity(tfidf[-1], tfidf)

  #Print the similarity scores
  #print(vals)

  #Get the index of the most similar text/sentence to the users response
  try:
        idx = vals.argsort()[0][-2]
  except IndexError :
        robo_response = robo_response+"I apologize, I don't understand."
        return robo_response

  #Reduce the dimensionality of vals
  flat = vals.flatten()

  #sort the list in ascending order
  flat.sort()

  #Get the most similar score to the users response
  score = flat[-2]

  #Print the similarity score
  #print(score)

  #If the variable 'score' is 0 then their is no text similar to the users response
  if(score == 0):
    robo_response = robo_response+"I apologize, I don't understand."
  else:
    robo_response = robo_response+sent_tokens[idx]
  
  #Print the chat bot response
  #print(robo_response)
  
  #Remove the users response from the sentence tokens list
  sent_tokens.remove(user_response)
  
  return robo_response


In [13]:
flag = True
print("SenseGrass: Hi! I will answer your queries.Please Ask. If you want to exit, type Bye!")
while(flag == True):
  user_response = input()
  user_response = user_response.lower()
  if(user_response != 'bye'):
    if(user_response == 'thanks' or user_response =='thank you'):
      flag=False
      print("SenseGrass: You are welcome !")
    else:
      if(greeting(user_response) != None):
        print("SenseGrass: "+greeting(user_response))
      else:
        print("SenseGrass: "+response(user_response))       
  else:
    flag = False
    print("SenseGrass: Chat with you later !")

SenseGrass: Hi! I will answer your queries.Please Ask. If you want to exit, type Bye!
Hi
SenseGrass: howdy
what are the Ions that contribute to soil salinity ?
SenseGrass: Ions that contribute to soil salinity include Cl-, SO,?-, HCO,-, Nat, Ca2+, Mg?t,
and, rarely, NO; or Kt.
what are nutritional effects of salinity ?
SenseGrass: Nutritional Effects

SALINITY ANDCATIONNUTRITION The major nutritional effects of salinity are
those associated with cation nutrition.
thanks
SenseGrass: You are welcome !
